<a href="https://colab.research.google.com/github/pensiun12/Sender_3F_WeatherPredict/blob/main/cuaca1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/ramalan

/content/drive/MyDrive/ramalan


In [ ]:
!pip install pandas scikit-learn joblib nltk requests

import pandas as pd
import numpy as np
import nltk
import datetime
import re
import joblib
import requests

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, mean_absolute_error

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import requests
import pandas as pd
import time

API_KEY = "c5a26176501047528ea10759250405"  # Ganti dengan API Key kamu
cities = [
    "Jakarta", "Surabaya", "Bandung", "Medan", "Makassar",
    "Kuala Lumpur", "Singapore", "Tokyo", "London", "New York",
    "Sydney", "Paris", "Cairo", "Los Angeles", "Berlin",
    "Dubai", "Bangkok", "Moscow", "Cape Town", "Toronto"
]

def fetch_forecast(city):
    url = f"http://api.weatherapi.com/v1/forecast.json?key={API_KEY}&q={city}&days=7&aqi=no&alerts=no"
    res = requests.get(url)
    if res.status_code != 200:
        print(f"Gagal ambil data untuk {city}: {res.status_code}")
        return None
    data = res.json()
    records = []
    for day in data['forecast']['forecastday']:
        for hour_data in day['hour']:
            records.append({
                'location': city,
                'datetime': hour_data['time'],
                'temp_c': hour_data['temp_c'],
                'condition': hour_data['condition']['text'],
                'humidity': hour_data['humidity'],
                'wind_kph': hour_data['wind_kph'],
                'pressure_mb': hour_data['pressure_mb'],
                'chance_of_rain': hour_data.get('chance_of_rain', 0),
                'lat': data['location']['lat'],
                'lon': data['location']['lon']
            })
    return pd.DataFrame(records)

# Proses semua kota
all_data = []
for city in cities:
    df_city = fetch_forecast(city)
    if df_city is not None:
        all_data.append(df_city)
    time.sleep(1)  # Delay agar tidak terlalu cepat request

# Gabungkan semua data
df_all = pd.concat(all_data, ignore_index=True)

# Simpan ke CSV
df_all.to_csv("cuaca_banyak_kota.csv", index=False)
print("Selesai ambil data. File tersimpan: cuaca_banyak_kota.csv")


Selesai ambil data. File tersimpan: cuaca_banyak_kota.csv


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load data
df = pd.read_csv("cuaca_banyak_kota.csv")

# Convert datetime
df['datetime'] = pd.to_datetime(df['datetime'])
df['hour'] = df['datetime'].dt.hour
df['dayofweek'] = df['datetime'].dt.dayofweek
df['month'] = df['datetime'].dt.month

# Encode kondisi cuaca
le = LabelEncoder()
df['condition_encoded'] = le.fit_transform(df['condition'])

# Fitur
features = ['lat', 'lon', 'hour', 'dayofweek', 'month', 'humidity', 'wind_kph', 'pressure_mb', 'chance_of_rain']
X = df[features]
y_temp = df['temp_c']
y_cond = df['condition_encoded']

# Normalisasi
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Simpan scaler & encoder
import joblib
joblib.dump(scaler, "scaler.pkl")
joblib.dump(le, "label_encoder.pkl")

print("Preprocessing selesai.")


Preprocessing selesai.


In [ ]:
API_KEY = "c5a26176501047528ea10759250405"


In [ ]:
import numpy as np

sequence_length = 24

def create_sequences(X, y):
    Xs, ys = [], []
    for i in range(len(X) - sequence_length):
        Xs.append(X[i:i+sequence_length])
        ys.append(y[i+sequence_length])
    return np.array(Xs), np.array(ys)

X_temp_seq, y_temp_seq = create_sequences(X_scaled, y_temp.values)
X_cond_seq, y_cond_seq = create_sequences(X_scaled, y_cond.values)

print(f"Shape X_temp_seq: {X_temp_seq.shape}, y_temp_seq: {y_temp_seq.shape}")


Shape X_temp_seq: (3336, 24, 9), y_temp_seq: (3336,)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import joblib

# 🔹 Ambil data cuaca dari banyak kota
# Contoh pakai file CSV (bisa ganti API/webscraping jika mau)
df = pd.read_csv('/content/drive/MyDrive/ramalan/cuaca_banyak_kota.csv')  # pastikan ada kolom: location, datetime, temp_c, condition, humidity, wind_kph, pressure_mb, chance_of_rain

# 🔹 Fitur & label
# Menyesuaikan dengan kolom yang ada
feature_cols = ['temp_c', 'humidity', 'pressure_mb', 'wind_kph', 'chance_of_rain']
X = df[feature_cols]
y_temp = df['temp_c']  # Target suhu
y_cond = df['condition']  # Target kondisi cuaca

# 🔹 Standarisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 🔹 Encode kondisi cuaca (seperti 'Sunny', 'Cloudy', dll.)
le = LabelEncoder()
y_cond_enc = le.fit_transform(y_cond)

# 🔹 Buat data sequence untuk LSTM
sequence_length = 24  # Kita buat sequence 24 jam sebelumnya untuk prediksi

def create_sequences(X, y):
    Xs, ys = [], []
    for i in range(len(X) - sequence_length):
        Xs.append(X[i:i+sequence_length])
        ys.append(y[i+sequence_length])  # Prediksi nilai suhu dan kondisi berikutnya
    return np.array(Xs), np.array(ys)

# Buat sequences untuk suhu dan kondisi
X_seq, y_temp_seq = create_sequences(X_scaled, y_temp.values)
_, y_cond_seq = create_sequences(X_scaled, y_cond_enc)

# 🔹 One-hot encode kondisi cuaca
y_cond_cat = to_categorical(y_cond_seq)

# ================= MODEL SUHU ===================
model_temp = Sequential([
    LSTM(128, input_shape=(X_seq.shape[1], X_seq.shape[2])),  # Layer LSTM
    Dropout(0.3),  # Regularisasi untuk menghindari overfitting
    Dense(64, activation='relu'),  # Fully connected layer
    Dense(1)  # Output layer untuk suhu (1 nilai per prediksi)
])

model_temp.compile(optimizer='adam', loss='mae', metrics=['mae'])
model_temp.fit(X_seq, y_temp_seq, epochs=100, batch_size=32, validation_split=0.1)

model_temp.save("model_suhu.keras")  # Simpan model suhu

# ================= MODEL KONDISI ===================
model_cond = Sequential([
    LSTM(128, input_shape=(X_seq.shape[1], X_seq.shape[2])),  # Layer LSTM
    Dropout(0.3),  # Regularisasi
    Dense(64, activation='relu'),  # Fully connected layer
    Dense(y_cond_cat.shape[1], activation='softmax')  # Output layer untuk kondisi cuaca (softmax untuk klasifikasi)
])

model_cond.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_cond.fit(X_seq, y_cond_cat, epochs=100, batch_size=32, validation_split=0.1)

model_cond.save("model_kondisi.keras")  # Simpan model kondisi

# 🔹 Simpan scaler & label encoder
joblib.dump(scaler, "scaler.save")  # Simpan scaler untuk standarisasi input di masa depan
joblib.dump(le, "label_encoder.save")  # Simpan label encoder untuk mengubah prediksi kondisi kembali ke label asli

print("✅ Model & scaler tersimpan")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


94/94 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step - loss: 14.9778 - mae: 14.9778 - val_loss: 2.7845 - val_mae: 2.7845
Epoch 2/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - loss: 4.0560 - mae: 4.0560 - val_loss: 2.4882 - val_mae: 2.4882
Epoch 3/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - loss: 3.0112 - mae: 3.0112 - val_loss: 1.8715 - val_mae: 1.8715
Epoch 4/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 2.4790 - mae: 2.4790 - val_loss: 2.0150 - val_mae: 2.0150
Epoch 5/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - loss: 2.1423 - mae: 2.1423 - val_loss: 1.6068 - val_mae: 1.6068
Epoch 6/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 1.9419 - mae: 1.9419 - val_loss: 1.3712 - val_mae: 1.3712
Epoch 7/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 1.8208 - mae: 1.8208 - val_loss: 1.2933 - val_mae: 1.2933
Epoch 8/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - loss: 1.7070 - mae: 1.7070 - val_loss: 1.4495 - val_mae: 1.4495
Epoch 9/100
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - loss: 1.6821

In [ ]:
print(df.columns)


Index(['location', 'datetime', 'temp_c', 'condition', 'humidity', 'wind_kph',
       'pressure_mb', 'chance_of_rain', 'lat', 'lon'],
      dtype='object')


In [3]:
import datetime
import numpy as np
import joblib
import requests
from tensorflow.keras.models import load_model
import re

# 🔹 Load model & scaler
model_temp = load_model("model_suhu.keras")
model_cond = load_model("model_kondisi.keras")
scaler = joblib.load("scaler.save")
le = joblib.load("label_encoder.save")

# 🔹 API key WeatherAPI
API_KEY = "c5a26176501047528ea10759250405"  # Ganti dengan API key Anda dari WeatherAPI
BASE_URL = "http://api.weatherapi.com/v1/forecast.json"

# 🔹 Fungsi untuk ambil data cuaca per jam dari WeatherAPI
def ambil_data_cuaca(kota):
    # URL untuk mengambil forecast 7 hari ke depan
    url = f"{BASE_URL}?key={API_KEY}&q={kota}&hours=168"  # 168 jam untuk 7 hari ke depan
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        forecast = data["forecast"]["forecastday"]

        # Ambil data per jam: suhu, kelembaban, tekanan, angin, hujan
        cuaca_data = []
        for day in forecast:
            for hour in day["hour"]:
                cuaca_data.append([
                    hour["temp_c"],      # suhu
                    hour["humidity"],    # kelembaban
                    hour["pressure_mb"], # tekanan
                    hour["wind_kph"],    # kecepatan angin
                    hour["chance_of_rain"]  # kemungkinan hujan
                ])

        return np.array(cuaca_data)
    else:
        print("Error: Tidak dapat mengambil data cuaca")
        return None

# 🔹 Fungsi prediksi
def prediksi_cuaca(kota, tanggal, jam):
    data = ambil_data_cuaca(kota)
    if data is None or data.shape[0] < 24:
        return "Data 24 jam sebelumnya kurang untuk prediksi."

    # Menggunakan standar scaler untuk mengubah data
    X_input = scaler.transform(data)
    X_input = np.expand_dims(X_input, axis=0)  # (1, 168, 5) <-- Pastikan hanya ada 5 fitur per jam

    # Prediksi suhu dan kondisi cuaca
    suhu_pred = model_temp.predict(X_input)[0][0]
    kondisi_pred = model_cond.predict(X_input)
    kondisi_label = le.inverse_transform([np.argmax(kondisi_pred)])[0]

    return f"Prediksi cuaca di {kota.capitalize()} tanggal {tanggal} jam {jam}: suhu {suhu_pred:.1f}°C, kondisi {kondisi_label}"

# 🔹 Chatbot
print("===== Chatbot Prediksi Cuaca AI =====")
print("Contoh: 'cuaca di Jakarta hari ini jam 15'")
print("Ketik 'exit' untuk keluar.")

while True:
    tanya = input("\n🤖 Kamu: ").lower()
    if tanya == 'exit':
        break

    lokasi = re.search(r'di (\w+)', tanya)
    jam = re.search(r'jam (\d+)', tanya)
    if lokasi and jam:
        kota = lokasi.group(1)
        jam = int(jam.group(1))
        tanggal = datetime.date.today().strftime('%Y-%m-%d')
        jawaban = prediksi_cuaca(kota, tanggal, jam)
        print(f"\n🤖 {jawaban}")
    else:
        print("\n🤖 Maaf, saya tidak mengerti. Coba contoh: 'cuaca di Bandung jam 10'")



===== Chatbot Prediksi Cuaca AI =====
Contoh: 'cuaca di Jakarta hari ini jam 15'
Ketik 'exit' untuk keluar.

🤖 Kamu: exit


In [4]:
!git config --global user.email '$ferrypratama9573@gmail.com'

In [ ]:
from google.colab import files
files.download("model_suhu.keras")
files.download("model_kondisi.keras")
files.download("scaler.pkl")
files.download("label_encoder.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>